In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Math, Latex
from datetime import datetime
import threading

In [2]:
#Read in Tickers
df = pd.read_csv("Tickers.csv")

In [3]:
#If column name is a ticker, we make the column name into a ticker
first_ticker = df.columns[0]

In [4]:
#Rename Columns Properly
df.rename(columns = {first_ticker:"ticker"},inplace=True)

In [5]:
#Filter based on if regularMarketPrice exists for a certain stock or not
ab = yf.Ticker(first_ticker)
if ab.info["regularMarketPrice"] != None:
    df = df.append({"ticker":first_ticker},ignore_index=True)

In [6]:
#Print filtered Dataframe
df

,ticker
0,ABBV
1,ABT
2,ACN
3,AGN
4,AIG
...,...
57,UNH
58,UNP
59,UPS
60,USB


In [7]:
#Get length of dataframe
length = len(df)

In [8]:
if yf.Ticker("AAPL").history(start="2017-01-01").Close.index.size == 0:
    print("a")
else:
    print("b")

b


In [9]:
data = yf.download(  # or pdr.get_data_yahoo(...
        tickers = list(df.ticker),
        period = "5y",
        group_by = 'ticker',
        auto_adjust = True,
        threads = True,
        proxy = None)

[*********************100%***********************]  62 of 62 completed

4 Failed downloads:
- CELG: No data found, symbol may be delisted
- AGN: No data found, symbol may be delisted
- PCLN: No data found for this date range, symbol may be delisted
- RTN: No data found, symbol may be delisted


In [10]:
inf = {}
l = 0
def get_info(ticker,start,end):
    global l
    for tick in ticker[start:end]:
        info = yf.Ticker(tick).info
        inf[tick] = info
        l+=100/length
        print("Completed ",round(l,2),"%",end="\r")

In [11]:
def split_processing(items, num_splits=10,split_size=100):                                      
    try:
        split_size = len(items) // num_splits 
    except:
        pass
    threads = []                                                                
    for i in range(num_splits):                                                 
        # determine the indices of the list this thread will handle             
        start = i * split_size                                                  
        # special case on the last chunk to account for uneven splits           
        end = None if i+1 == num_splits else (i+1) * split_size                 
        # create the thread                                                     
        threads.append(                                                         
            threading.Thread(target=get_info, args=(items,start,end)))         
        threads[-1].start() # start the thread we just created                  

    # wait for all threads to finish                                            
    for t in threads:                                                           
        t.join()                                                                


In [12]:
split_processing(df.ticker)

In [13]:
#Check for delisted and non-existent stocks
start_date = "2017-01-01"
dict2 = {}
pct = 0
global inf
for i in df.ticker:
    info = inf[i]
    a = data[i][data[i].index >= datetime.fromisoformat(start_date)]
    if a.Close.index.size > 0:
        dict2[i] = {"info":info,"price":a}
    pct+=100/length
    print("Completed", round(pct,2),"%",end="\r")

In [14]:
# #Check for delisted and non-existent stocks
# start_date = "2017-01-01"
# dict2 = {}
# pct = 0
# for i in df.ticker:
#     y = yf.Ticker(i)
#     inf = y.info
#     a = y.history(start=start_date)
#     if a.Close.index.size > 0:
#         dict2[i] = {"info":inf,"price":a}
#     pct+=100/length
#     print("Completed", round(pct,2),"%",end="\r")

In [15]:
#Filter stocks based on currency and average share volume
filtered_stock_data = {}
for i,j in dict2.items():
    try:
        if j["info"]["currency"] == "USD" and np.mean(j["price"]["Volume"]) > 10000:
            filtered_stock_data[i]=j
        else:
            pass
    except KeyError:
        pass
len(filtered_stock_data)

55

In [16]:
#Calculate std of each stock from dictionary
def std_stock(stock):
    close=stock['price']['Close']
    std=close.pct_change().std()
    return std

In [17]:
#Create new dictionary with Ticker & STD of ticker
dict3={}
for i,j in filtered_stock_data.items():
    dict3[i]= std_stock(j)
    
dict3

{'ABBV': 0.01755991241200861,
 'ABT': 0.01556600891581009,
 'ACN': 0.015488419317235292,
 'AIG': 0.023951826719523464,
 'AMZN': 0.01845392899466147,
 'AXP': 0.02146437205014576,
 'BA': 0.029437585783863327,
 'BAC': 0.021173759036807363,
 'BIIB': 0.029550362272822695,
 'BK': 0.018353459603313152,
 'BLK': 0.017997544324245544,
 'BMY': 0.01546687507100411,
 'C': 0.022991058372361522,
 'CAT': 0.019562965767500353,
 'CL': 0.012969903654895913,
 'CMCSA': 0.01667278165078533,
 'COF': 0.024739114780443532,
 'COP': 0.026235158302871427,
 'COST': 0.013298454416953553,
 'CSCO': 0.016953064176207587,
 'CVS': 0.01769890375641562,
 'GM': 0.02375464103507434,
 'GOOG': 0.01690841382696549,
 'JPM': 0.01887996656778318,
 'KMI': 0.01969301919124308,
 'KO': 0.012541469736895185,
 'LLY': 0.01742911689056282,
 'LMT': 0.0155583356432705,
 'MO': 0.01592345590268477,
 'MON': 0.004594132392755029,
 'MRK': 0.01406261374288112,
 'MS': 0.021176801116526638,
 'MSFT': 0.017172178573694862,
 'NEE': 0.0149214024438445

In [18]:
#We create a function to sort and get the 15 least std values with keys
#n smallest dictionsary
list_nsmallest = []
temp_dict = dict3.copy()
while True:
    temp = min(temp_dict.values())
    res = [key for key in temp_dict if temp_dict[key] == temp]
    list_nsmallest.append(res[0])
    temp_dict.pop(res[0])
    if len(list_nsmallest) == 15:
        break

In [19]:
res=list_nsmallest

In [20]:
res

['MON',
 'KO',
 'PG',
 'CL',
 'PEP',
 'COST',
 'MRK',
 'PFE',
 'T',
 'NEE',
 'BMY',
 'ACN',
 'SO',
 'LMT',
 'ABT']

In [21]:
#Function to make pairs of 2 tickers for correlation checking
def combinations(lst, depth, start=0, prepend=[]):
    if depth <= 0:
        yield prepend
    else:
        for i in range(start, len(lst)):
            for c in combinations(lst, depth - 1, i + 1, prepend + [lst[i]]):
                yield c

In [22]:
comb = combinations(res, 10)

In [23]:
#Function that checks correlation of all pairs and outputs a list of least correlated stocks so portfolio is as less volatile as possible
n=0
min1=10
a=[]
ans=None
comb_length = 3002
for i in list(comb):
    for j in range(len(i)):
        b=[]
        if j == len(i)-1:
            monthly_returns_1 = dict2[i[j]]['price']['Close'].resample('MS').first()
            monthly_returns_2 = dict2[i[0]]['price']['Close'].resample('MS').first()
            b.append(pd.concat([monthly_returns_1,monthly_returns_2],join="inner",axis=1).corr().iat[0,1])
        else:
            monthly_returns_1 = dict2[i[j]]['price']['Close'].resample('MS').first().pct_change()
            monthly_returns_2 = dict2[i[j+1]]['price']['Close'].resample('MS').first().pct_change()
            b.append(pd.concat([monthly_returns_1,monthly_returns_2],join="inner",axis=1).corr().iat[0,1])
    avg = sum(b)/10
    if avg <= min1:
        min1 = avg
        n+=100/ comb_length
#         print(min1, i, n)
        print("Completed", round(n,2),"%",end="\r")
        ans=i
    else:
        n+=100/comb_length
        print("Completed", round(n,2),"%",end="\r")
#         print(avg, n)
    a.append(avg)
ans

['MON', 'PG', 'CL', 'PEP', 'COST', 'MRK', 'PFE', 'T', 'NEE', 'BMY']

In [24]:
ans
dict_with_std = {}
for i in ans:
    dict_with_std[i] = dict3[i]
sorted_x = dict(sorted(dict_with_std.items(), key=lambda x: x[1]))
sorted_final_list = list(sorted_x.keys())
sorted_final_list

['MON', 'PG', 'CL', 'PEP', 'COST', 'MRK', 'PFE', 'T', 'NEE', 'BMY']

In [25]:
#Creating date variables
starting='2017-01-01'
now=datetime.now()
buy_on_date = now.strftime("%Y-%m-%d")

#Dictionary of the stock to stock price data
stock_prices={}

for i in (range(0,len(sorted_final_list))):
    stock_prices[sorted_final_list[i]]=pd.DataFrame(yf.Ticker(sorted_final_list[i]).history(start=starting,end=buy_on_date).Close)
    stock_prices[sorted_final_list[i]].columns=[sorted_final_list[i]+" Closing"]

# #Creating stock price dataframe
index_dataframe=stock_prices[sorted_final_list[0]]
for i in (range(1,len(sorted_final_list))):
    index_dataframe=pd.concat([index_dataframe,stock_prices[sorted_final_list[i]]],join='inner',axis=1)

#Output dataframe of closing prices
index_dataframe

,MON Closing,PG Closing,CL Closing,PEP Closing,COST Closing,MRK Closing,PFE Closing,T Closing,NEE Closing,BMY Closing
Date,,,,,,,,,,
2021-03-16,9.785,126.535484,74.579483,133.038055,325.939514,72.021919,34.855972,27.859243,74.312981,60.477669
2021-03-17,9.680,126.025169,74.520500,132.650879,327.871735,72.481377,34.817055,28.007877,71.927238,60.751232
2021-03-18,9.770,126.299950,74.785942,131.568787,321.686615,72.453247,34.797600,27.877823,70.123146,60.897789
2021-03-19,9.750,125.622818,73.714340,133.524506,327.592865,72.678284,34.564121,27.645584,69.837242,61.259285
2021-03-22,9.700,127.752342,74.490997,136.800568,333.150513,72.678284,35.021347,27.859243,70.675224,61.923664
...,...,...,...,...,...,...,...,...,...,...
2021-11-18,9.740,147.119995,76.389999,163.419998,529.369995,83.050003,51.410000,24.389999,87.213722,59.040001
2021-11-19,9.760,146.820007,77.199997,163.809998,533.789978,80.699997,50.799999,24.129999,87.920593,57.830002
2021-11-22,9.750,147.800003,77.559998,164.149994,539.650024,81.639999,51.200001,24.700001,86.974785,57.040001


In [26]:
#Computing daily returns using log change
#This means a log change of +0.1 today and then -0.1 tomorrow will give you the same value of stock as yesterday. 
#This is not true if you simply compute percentage change.
daily_return = index_dataframe.pct_change().apply(lambda x: np.log(1+x))
daily_return=daily_return.iloc[1: ,:]

#print daily returns
daily_return

,MON Closing,PG Closing,CL Closing,PEP Closing,COST Closing,MRK Closing,PFE Closing,T Closing,NEE Closing,BMY Closing
Date,,,,,,,,,,
2021-03-17,-0.010789,-0.004041,-0.000791,-0.002915,0.005911,0.006359,-0.001117,0.005321,-0.032631,0.004513
2021-03-18,0.009255,0.002178,0.003556,-0.008191,-0.019045,-0.000388,-0.000559,-0.004654,-0.025402,0.002410
2021-03-19,-0.002049,-0.005376,-0.014433,0.014755,0.018194,0.003101,-0.006732,-0.008365,-0.004086,0.005919
2021-03-22,-0.005141,0.016810,0.010481,0.024239,0.016823,0.000000,0.013142,0.007699,0.011928,0.010787
2021-03-23,-0.004132,0.018419,0.025796,0.011974,0.017338,-0.016127,-0.017938,0.000333,0.031984,-0.012064
...,...,...,...,...,...,...,...,...,...,...
2021-11-18,0.000000,0.000136,-0.015458,0.000306,0.005835,0.005433,0.010559,-0.011009,-0.002053,-0.006752
2021-11-19,0.002051,-0.002041,0.010548,0.002384,0.008315,-0.028704,-0.011936,-0.010717,0.008072,-0.020707
2021-11-22,-0.001025,0.006653,0.004652,0.002073,0.010918,0.011581,0.007843,0.023347,-0.010816,-0.013755


In [27]:
# Log of percentage change
cov_matrix = index_dataframe.pct_change().apply(lambda x: np.log(1+x)).cov()

#Print covariance matrix
cov_matrix

,MON Closing,PG Closing,CL Closing,PEP Closing,COST Closing,MRK Closing,PFE Closing,T Closing,NEE Closing,BMY Closing
MON Closing,2.148807e-05,9.851117e-07,0.000002,0.000004,0.000001,9.549868e-07,4.071845e-06,-0.000001,-0.000005,-0.000002
PG Closing,9.851117e-07,6.486416e-05,0.000044,0.000045,0.000035,1.678234e-05,2.653835e-05,0.000019,0.000038,0.000023
CL Closing,1.872308e-06,4.396184e-05,0.000077,0.000042,0.000023,1.784846e-05,2.221370e-05,0.000029,0.000041,0.000022
PEP Closing,3.961710e-06,4.532474e-05,0.000042,0.000067,0.000040,1.727402e-05,2.931078e-05,0.000020,0.000041,0.000024
COST Closing,1.409093e-06,3.469089e-05,0.000023,0.000040,0.000103,1.713686e-05,1.822511e-05,0.000014,0.000039,0.000014
MRK Closing,9.549868e-07,1.678234e-05,0.000018,0.000017,0.000017,2.337114e-04,3.914123e-08,0.000012,0.000007,0.000073
PFE Closing,4.071845e-06,2.653835e-05,0.000022,0.000029,0.000018,3.914123e-08,2.311784e-04,0.000030,0.000021,0.000063
T Closing,-1.165913e-06,1.908869e-05,0.000029,0.000020,0.000014,1.211380e-05,3.037121e-05,0.000109,0.000021,0.000019
NEE Closing,-5.398168e-06,3.849895e-05,0.000041,0.000041,0.000039,7.299098e-06,2.055826e-05,0.000021,0.000132,0.000018
BMY Closing,-2.163746e-06,2.310244e-05,0.000022,0.000024,0.000014,7.300308e-05,6.341670e-05,0.000019,0.000018,0.000107


In [28]:
#Average Yearly Returns For Individual Companies 
yr_er=daily_return.resample('Y').last().mean()

#Print returns
yr_er

MON Closing     0.000000
PG Closing     -0.005233
CL Closing     -0.002184
PEP Closing    -0.009180
COST Closing    0.008164
MRK Closing    -0.006300
PFE Closing    -0.003727
T Closing      -0.011782
NEE Closing     0.003273
BMY Closing    -0.011203
dtype: float64

In [29]:
portfolio_returns=[] # Empty Array for Portfolio Returns
portfolio_volatility=[] #Empty Array for Portfolio Volatility
portfolio_weights=[] #Empty Array for Asset Weights

num_assets=len(daily_return.columns)
num_portfolios=10000 #Create as much portfolios as possible

In [30]:
# This for-loop computers the returns,volatilty and invidual weights for each portfolio
for portfolio in range(num_portfolios):
    weights=np.random.random(num_assets)
    weights=weights/np.sum(weights)
    portfolio_weights.append(weights)
    returns=np.dot(weights,yr_er) #Expected returns are product of individual expected returns x weights
    portfolio_returns.append(returns)
    var=cov_matrix.mul(weights,axis=0).mul(weights,axis=1).sum().sum() #Portfolio Variance
    sd=np.sqrt(var) # Daily Standard Deviation
    ann_sd=sd*np.sqrt(175) # Annual Standard Deviation = volatility (used 175 thats because thats the # of range of our data)(should be 250 but we don't even have a full year of expected returns)
    portfolio_volatility.append(ann_sd)

In [31]:
data={'Returns':portfolio_returns,'Volatility':portfolio_volatility}

for counter, symbol in enumerate(daily_return.columns.tolist()):
    data[symbol+' weight']=[w[counter] for w in portfolio_weights]

In [32]:
portfolios=pd.DataFrame(data)
portfolios #Dataframe of the 10k portfolios created

,Returns,Volatility,MON Closing weight,PG Closing weight,CL Closing weight,PEP Closing weight,COST Closing weight,MRK Closing weight,PFE Closing weight,T Closing weight,NEE Closing weight,BMY Closing weight
0,-0.005544,0.070275,0.174333,0.060155,0.199042,0.165146,0.003499,0.049443,0.084992,0.135230,0.024149,0.104012
1,-0.003240,0.073105,0.146362,0.152293,0.056353,0.070485,0.156725,0.028380,0.095795,0.027488,0.061371,0.204748
2,-0.005801,0.072692,0.152184,0.119038,0.101360,0.089896,0.002928,0.152144,0.012621,0.163219,0.075128,0.131482
3,-0.002964,0.079007,0.106816,0.061853,0.075718,0.078894,0.098120,0.051064,0.199232,0.154695,0.157491,0.016118
4,-0.001411,0.075427,0.164262,0.194870,0.025163,0.079598,0.214831,0.181879,0.007525,0.032346,0.090537,0.008989
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-0.006759,0.086974,0.050509,0.029957,0.169893,0.129152,0.013074,0.201223,0.043867,0.094932,0.027142,0.240252
9996,-0.002556,0.072905,0.157762,0.039610,0.051883,0.168382,0.121886,0.124885,0.071291,0.026057,0.161924,0.076320
9997,-0.003740,0.076091,0.113079,0.122740,0.173382,0.051159,0.055915,0.146692,0.132480,0.062204,0.071806,0.070544
9998,-0.003509,0.070976,0.142892,0.158733,0.081945,0.046764,0.140840,0.147401,0.033167,0.167764,0.049066,0.031427


In [33]:
#Filter dataframe so it fits the weight restrictions of min of (100/2n) & and max of 35%
filterinfDataframe = portfolios[(portfolios[portfolios.columns[2]] >= 0.05) & (portfolios[portfolios.columns[2]] <= 0.35) &
                                (portfolios[portfolios.columns[3]] >= 0.05) & (portfolios[portfolios.columns[3]] <= 0.35) &
                                (portfolios[portfolios.columns[4]] >= 0.05) & (portfolios[portfolios.columns[4]] <= 0.35) &
                                (portfolios[portfolios.columns[5]] >= 0.05) & (portfolios[portfolios.columns[5]] <= 0.35) &
                                (portfolios[portfolios.columns[6]] >= 0.05) & (portfolios[portfolios.columns[6]] <= 0.35) &
                                (portfolios[portfolios.columns[7]] >= 0.05) & (portfolios[portfolios.columns[7]] <= 0.35) &
                                (portfolios[portfolios.columns[8]] >= 0.05) & (portfolios[portfolios.columns[8]] <= 0.35) &
                                (portfolios[portfolios.columns[9]] >= 0.05) & (portfolios[portfolios.columns[9]] <= 0.35) &
                                (portfolios[portfolios.columns[10]] >= 0.05) & (portfolios[portfolios.columns[10]] <= 0.35) &
                                (portfolios[portfolios.columns[11]] >= 0.05) & (portfolios[portfolios.columns[11]] <= 0.35)
                               ]
#Reset index to start from 0
filterinfDataframe.reset_index(inplace=True)
#Print dataframe
filterinfDataframe

,index,Returns,Volatility,MON Closing weight,PG Closing weight,CL Closing weight,PEP Closing weight,COST Closing weight,MRK Closing weight,PFE Closing weight,T Closing weight,NEE Closing weight,BMY Closing weight
0,83,-0.003145,0.073064,0.134258,0.095306,0.058459,0.069847,0.126984,0.154849,0.090739,0.091176,0.101709,0.076673
1,87,-0.003780,0.072486,0.139797,0.107977,0.129409,0.078224,0.092202,0.084178,0.125423,0.069958,0.054620,0.118211
2,111,-0.003422,0.067969,0.165732,0.131455,0.097777,0.072868,0.104798,0.051952,0.072097,0.142951,0.094628,0.065743
3,129,-0.002997,0.078307,0.053383,0.085910,0.113851,0.077008,0.143979,0.067587,0.101026,0.071343,0.143569,0.142344
4,161,-0.003951,0.073510,0.089571,0.133706,0.136554,0.085445,0.095991,0.100882,0.063810,0.139056,0.089462,0.065525
...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,9906,-0.003337,0.075722,0.074901,0.092340,0.129165,0.098818,0.112788,0.090486,0.107371,0.111314,0.120508,0.062309
295,9913,-0.003085,0.073835,0.106380,0.125279,0.055506,0.093269,0.163202,0.081886,0.097762,0.073609,0.085563,0.117544
296,9943,-0.004406,0.076366,0.097702,0.092257,0.128966,0.127753,0.091281,0.155124,0.053059,0.061159,0.058038,0.134661
297,9959,-0.003691,0.071387,0.131230,0.087810,0.054370,0.104200,0.131128,0.080967,0.082377,0.101974,0.091388,0.134555


In [34]:
#Filter dataframe to get the portfolio with lowest volatility from the above dataframe
min_vol_portfolio=filterinfDataframe.iloc[filterinfDataframe['Volatility'].idxmin()]
#drop the volatility,returns and extra index columns
all_weights=min_vol_portfolio.drop(min_vol_portfolio.index[[0,1,2]])
#output weights
all_weights

MON Closing weight     0.240386
PG Closing weight      0.067002
CL Closing weight      0.064044
PEP Closing weight     0.150427
COST Closing weight    0.129273
MRK Closing weight     0.073297
PFE Closing weight     0.053025
T Closing weight       0.068490
NEE Closing weight     0.085350
BMY Closing weight     0.068707
Name: 243, dtype: float64

In [35]:
#Capital 
starting_money = 100000
#For calculating range
num_of_stocks=len(all_weights)
#output number of stocks
num_of_stocks

10

In [36]:
# This for-loop computers the shares of each ticker based on the weights and capital computed and given
final_shares_dict={}
for i in range(num_of_stocks):
    #Remeber to change '2021-11-24' to '2021-11-26' before submitting
    shares=(starting_money*all_weights[i])/(filtered_stock_data[ans[i]]["price"]["Close"]['2021-11-24'])
    final_shares_dict[ans[i]]=shares
    
#Output Tickers & Shares
final_shares_dict

{'MON': 2450.415600070281,
 'PG': 45.07034452454446,
 'CL': 82.36149844749013,
 'PEP': 91.8693390411753,
 'COST': 23.515713050007765,
 'MRK': 89.08221551432473,
 'PFE': 104.19493156838777,
 'T': 279.8953938105648,
 'NEE': 97.86665847111313,
 'BMY': 120.94172608754914}

In [37]:
#Store tickers
ticker=final_shares_dict.keys()
#Store the closing prices of each stock on '2021-11-26' (remeber to change price value to '2021-11-26')
price=[filtered_stock_data[ans[i]]["price"]["Close"]['2021-11-24'] for i in range(num_of_stocks)]
# convert the shares into a list
shares=list(final_shares_dict.values())
#Compute value of allocation of each stock by price*shares
value=[a*b for a,b in zip(price,shares)]
#List of weights
weight=list(all_weights)

In [38]:
#Create a dataframe that holds the ticker,price,shares,value & weight
FinalPortfolio=pd.DataFrame([ticker,price,shares,value,weight]).T
#start index from 1-10
FinalPortfolio.index+=1
#rename columns
FinalPortfolio.columns=['Ticker','Price','Shares','Value','Weight']

In [39]:
#Showing Weights & Portfolio Equal 100% and 100k Respectively 
total_weight=FinalPortfolio['Weight'].sum()
total_value=FinalPortfolio['Value'].sum()

In [40]:
#Printing proof of total weight summation
print("Total Weight Of Portfolio: ",total_weight)

Total Weight Of Portfolio:  1.0


In [41]:
#Print proof o total value of portfolio
print("Total Value Of Portfolio: ",total_value)

Total Value Of Portfolio:  99999.99999999997


In [42]:
#Print Final Portfolio
FinalPortfolio

,Ticker,Price,Shares,Value,Weight
1,MON,9.81,2450.4156,24038.578065,0.240386
2,PG,148.660004,45.070345,6700.157582,0.067002
3,CL,77.760002,82.361498,6404.430295,0.064044
4,PEP,163.740005,91.869339,15042.686079,0.150427
5,COST,549.72998,23.515713,12927.292476,0.129273
6,MRK,82.279999,89.082216,7329.684584,0.073297
7,PFE,50.889999,104.194932,5302.480004,0.053025
8,T,24.469999,279.895394,6849.040094,0.06849
9,NEE,87.209999,97.866658,8534.951196,0.08535
10,BMY,56.810001,120.941726,6870.699625,0.068707


In [44]:
#Create final dataframe to hold tickers and shares
Stocks = pd.DataFrame(final_shares_dict.values(),final_shares_dict.keys())
#reset index
Stocks.reset_index(inplace=True)
#start index from 1-10
Stocks.index+=1
#rename columns
Stocks.rename(columns={"index":"Ticker",0:"Shares"},inplace=True)
#output dataframe to csv file
Stocks.to_csv("Stocks_Group_07.csv")
#Print Tickers & Shares (Stocks Dataframe)
Stocks

,Ticker,Shares
1,MON,2450.415600
2,PG,45.070345
3,CL,82.361498
4,PEP,91.869339
5,COST,23.515713
6,MRK,89.082216
7,PFE,104.194932
8,T,279.895394
9,NEE,97.866658
10,BMY,120.941726
